# COMP5329 - Deep Learning
## Assignment 2 - Multilabel Classification

*   Nirav Desai
*   Rebecca Chan

## Initial Set up
### Load packages

In [ ]:
## Load libraries
import re, os
import pandas as pd
from io import StringIO
from google.colab import drive

%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import time
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import itertools
import os

from torchvision import datasets, transforms, models
from torch.utils.data import Dataset
from sklearn.metrics import confusion_matrix
from datetime import timedelta
from torch import optim
from sklearn.metrics import classification_report

torch.__version__

'1.8.1+cu101'

##### Mount google drive

In [ ]:
use_cuda = True if torch.cuda.is_available() else False
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('We are using GPU.' if use_cuda else 'We are using CPU.')


We are using GPU.


In [ ]:
drive.mount('/content/gdrives')

Mounted at /content/gdrives


### Select Drive Path for Teammates

**Values :** 
- rebecca : Rebecca's drive config
- nirav : Nirav's drive config

In [ ]:
## Input name to get correct drive path
name = input()
if name == 'rebecca':
  data_path = '/content/gdrives/MyDrive/COMP5329_attempt2'
elif name == 'nirav': 
  data_path = '/content/gdrives/MyDrive/COMP5329/COMP5329S1A2Dataset'

rebecca


## Load Dataset Function

**Inputs:**
  1. Root Directory for Images
  2. csv filename with annotations
  3. Pytorch transform object

**Outputs:**
  - Training and Validation Set
    - 2 different method per user (Rebecca and Nirav) due to drive paths
    - tuple with image as a tensor and corresponding target One-Hot encoded label as tensor (image, label)
  - Test Set
    - image as a tensor

In [ ]:
if name == 'nirav':
  class load_data(Dataset):
      def __init__(self, root_dir, annotation_file, transform=None):
          with open(root_dir+'/'+annotation_file) as train: 
            lines = [re.sub(r'([^,])"(\s*[^\n])', r'\1/"\2', line) for line in train]
            train_labels = pd.read_csv(StringIO(''.join(lines)), escapechar="/", encoding='utf-8')

          self.root_dir = root_dir
          self.annotations = train_labels
          self.transform = transform

      def __len__(self):
          return len(self.annotations)

      def __getitem__(self, index):
          img_id = self.annotations.iloc[index, 0]
          img = Image.open(os.path.join(self.root_dir+'/data', img_id)).convert("RGB")
          split_label = self.annotations.iloc[index, 1].split()
          num_labels = list(map(int, split_label))
          label = torch.LongTensor(num_labels) - 1

          # One Hot encoding Target labels
          one_hot_labels = torch.zeros(size=([19]), dtype=torch.float64)
          y_label = one_hot_labels.scatter_(dim=0, index=label, value=1.)

          if self.transform is not None:
              img = self.transform(img)

          return (img, y_label)

elif name == 'rebecca':
  class load_data(Dataset):
      def __init__(self, root_dir, annotation_file, transform=None):
          with open(root_dir+'/'+annotation_file) as train: 
            lines = [re.sub(r'([^,])"(\s*[^\n])', r'\1/"\2', line) for line in train]
            train_labels = pd.read_csv(StringIO(''.join(lines)), escapechar="/", encoding='utf-8')

          self.root_dir = root_dir
          self.annotations = train_labels
          self.transform = transform

      def __len__(self):
          return len(self.annotations)

      def __getitem__(self, index):
          img_id = self.annotations.iloc[index, 0]
          val_temp = val_temp = int(re.search('^\d+\.', img_id).group(0)[:-1])
          if val_temp < 5000: 
            folder_dir = 'folder_1'
          elif val_temp < 10000: 
            folder_dir = 'folder_2'
          elif val_temp < 15000: 
            folder_dir = 'folder_3'
          elif val_temp < 20000: 
            folder_dir = 'folder_4'
          elif val_temp < 25000: 
            folder_dir = 'folder_5'
          elif val_temp < 30000:
            folder_dir = 'folder_6'
          img = Image.open(os.path.join(self.root_dir+'/uploads/'+folder_dir, img_id)).convert("RGB")
          split_label = self.annotations.iloc[index, 1].split()
          num_labels = list(map(int, split_label))
          label = torch.LongTensor(num_labels) - 1
          one_hot_labels = torch.zeros(size=([19]), dtype=torch.float64)
          # print(label, self.one_hot_labels.shape, label.shape)
          y_label = one_hot_labels.scatter_(dim=0, index=label, value=1.)
          # y_label = torch.tensor(float(self.annotations.iloc[index, 1]))

          if self.transform is not None:
              img = self.transform(img)

          return (img, y_label)

if name == 'nirav':
  class load_test_data(Dataset):
      def __init__(self, root_dir, annotation_file, transform=None): 
          with open(root_dir + '/' + annotation_file) as test: 
            lines = [re.sub(r'([^,])"(\s*[^\n])', r'\1/"\2', line) for line in test]
            test_labels = pd.read_csv(StringIO(''.join(lines)), escapechar="/", encoding='utf-8')    

          self.root_dir = root_dir
          self.annotations = test_labels
          self.transform = transform
        
      def __len__(self):
          return len(self.annotations)

      def __getitem__(self, index):
          img_id = self.annotations.iloc[index,0]
          img = Image.open(os.path.join(self.root_dir+'/data', img_id)).convert("RGB")

          if self.transform is not None: 
              img = self.transform(img)

          return (img)

elif name == 'rebecca':
  class load_test_data(Dataset):
      def __init__(self, root_dir, annotation_file, transform=None): 
          with open(root_dir + '/' + annotation_file) as test: 
            lines = [re.sub(r'([^,])"(\s*[^\n])', r'\1/"\2', line) for line in test]
            test_labels = pd.read_csv(StringIO(''.join(lines)), escapechar="/", encoding='utf-8')    

          self.root_dir = root_dir
          self.annotations = test_labels
          self.transform = transform
        
      def __len__(self):
          return len(self.annotations)

      def __getitem__(self, index):
          img_id = self.annotations.iloc[index,0]
          img = Image.open(os.path.join(self.root_dir+'/test_data/test_set', img_id)).convert("RGB")

          if self.transform is not None: 
              img = self.transform(img)

          return (img)

## Image Pre-processing and Augmentation

- Augmentation Techniues such as:
  - Crop
  - Resize
  - Flipping (Horizontal and Vertical)
  - Rotation
  - Brightness Changes
- Normalize
- Load raw data and split into Training and Validation
- Create pytorch DataLoader instance for Train, Validation and Test set

In [ ]:
# https://stackoverflow.com/questions/56774582/adding-custom-labels-to-pytorch-dataloader-dataset-does-not-work-for-custom-data (for adding labels)
# https://medium.com/analytics-vidhya/implementing-cnn-in-pytorch-with-custom-dataset-and-transfer-learning-1864daac14cc (maybe better?)
# https://pytorch.org/tutorials/beginner/basics/data_tutorial.html try this

# DataLoader Parameters
batch_size = 32
num_workers = 2
pin_memory = True
shuffle=True

kwargs = {'num_workers': num_workers, 'pin_memory': pin_memory} if use_cuda else {}
kwargs['batch_size'] = batch_size  # Batch size used during training.

# Transform for data pre-processing
transform = transforms.Compose([transforms.Resize(240), 
                                transforms.CenterCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.RandomRotation(degrees=[-30, 30]),
                                transforms.ColorJitter(brightness=0.4),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.4914, 0.48216, 0.44653), (0.24703, 0.24349, 0.26159))
                                ])

test_transform = transforms.Compose([transforms.Resize(240), 
                                transforms.CenterCrop(224),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.4914, 0.48216, 0.44653), (0.24703, 0.24349, 0.26159))
                                ])

raw_dataset = load_data(data_path,"train.csv",transform=transform)
test_dataset = load_test_data(data_path, "test.csv", transform=test_transform)

# Train - Validation split
total_raw_data = len(raw_dataset)
train_valid_split = 0.2
valid_record_size = int(train_valid_split * total_raw_data)
train_record_size = int(total_raw_data - valid_record_size)
print(train_record_size, valid_record_size)

train_set, validation_set = torch.utils.data.random_split(raw_dataset, [train_record_size,valid_record_size])

train_loader = torch.utils.data.DataLoader(dataset=train_set, 
                                           shuffle=shuffle, 
                                           batch_size=batch_size,
                                           num_workers=num_workers,
                                           pin_memory=pin_memory)

validation_loader = torch.utils.data.DataLoader(dataset=validation_set, 
                                                shuffle=shuffle, 
                                                batch_size=batch_size,
                                                num_workers=num_workers, 
                                                pin_memory=pin_memory)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          shuffle = False, 
                                          batch_size = None
                                          )
print('Dataloaders initialized.')

images, labels = next(iter(train_loader))
print('Shape of an image batch:', images.size())

# Should be approx. 40,000 images (30,000 for training, 10,000 for testing)
print('No. images loaded: ',len(train_loader)*batch_size)

23997 5999
Dataloaders initialized.
Shape of an image batch: torch.Size([32, 3, 224, 224])
No. images loaded:  24000


## Exploratory Dataset Analysis

#### Training Labels

In [ ]:
training_indices = raw_dataset.annotations['Labels'][train_set.indices]
ti_split = training_indices.str.split()
t_counts = ti_split.explode().value_counts()
t_counts.index = t_counts.index.astype(int)
t_cnt = t_counts.sort_index()
print(t_cnt)
t_cnt = t_cnt.to_numpy()
total_cnt = np.insert(t_cnt,11,0) # for missing label 12, inserting 0



#### Training Label Distribution

In [ ]:
%matplotlib inline
class_labels = ['Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6', 'Class 7', 'Class 8', 'Class 9', 'Class 10', 'Class 11', 'Class 12', 'Class 13', 'Class 14', 'Class 15', 
'Class 16', 'Class 17', 'Class 18','Class 19']
plt.rcParams['figure.figsize'] = [8, 8]
plt.title('Training Data Label Distribution')
plt.ylabel('Count of Samples')
plt.xlabel('Training Label')
plt.xticks(np.arange(0,19,1), class_labels, rotation=45)
plt.bar(np.arange(len(total_cnt)),total_cnt, width=0.7)


### Functions and definitions necessary for evaluation

In [ ]:
def accuracy(predicted, original):  
    """
    Calculates the number of labels correctly predicted for each image
    """
    return torch.round(predicted).eq(original).sum().numpy()/len(original)

In [ ]:
## Create list of class names for matching in the evaluation and prediction stage
class_names = np.array(list(range(1, 20)))

## Modelling stage:
### Implement AlexNet

In [ ]:
class AlexNet(nn.Module):

    def __init__(self, num_classes=19, stem_stride=4):
        # invoke super class initialisation method
        super(AlexNet, self).__init__()
        
        # define the CNN:
        # 1. define feature extraction layers
        self.features = nn.Sequential(
            # conv-relu-pooling
            nn.Conv2d(3, 64, kernel_size=11, stride=stem_stride, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            # conv-relu-pooling
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            # conv-relu-conv-relu-conv-relu-pooling
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        # 2. define average pooling layer
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        # 3. define fully connected layers
        self.classifier = nn.Sequential(
            nn.Dropout(),                   # use dropout
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),                   # use dropout
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
            nn.Sigmoid(),
        )

    # define forward-propagation
    def forward(self, x):
        # feature extraction
        x = self.features(x)
        # adaptive pooling layer
        x = self.avgpool(x)
        # flatten feature map
        x = torch.flatten(x, 1)
        # classification
        x = self.classifier(x)
        return x

### Model Training with hypermarameters

In [ ]:
## Initialise and train an alexnet model
model_alexnet = AlexNet().to(dev)
criterion = nn.BCELoss()
optimizer = optim.Adam(model_alexnet.parameters(), lr = 3e-4, weight_decay = 0.01)

train_loss_tracker = []
train_accuracy_tracker = []
epoch_loss = []

epochs = 25
for epoch in range(epochs):
  print(f'Epoch no. {epoch + 1}')

  start = time.time()
  ep_loss = 0.0
  ep_acc = 0.0

  for i, data in enumerate(train_loader, 0):
    inputs, labels = data

    inputs = inputs.to(dev)
    labels = labels.to(dev)

    optimizer.zero_grad()

    outputs = model_alexnet(inputs)
    loss = criterion(outputs.float(), labels.float())
    ep_loss += loss

    acc = accuracy(outputs.to('cpu'), labels.to('cpu'))
    ep_acc += acc
    loss.backward()
    optimizer.step()
  
  train_loss_tracker.append(ep_loss/(i+1))
  train_accuracy_tracker.append(ep_acc/(i+1))
  print(f'Loss: {ep_loss/(i+1) :.3f}, Accuracy: {ep_acc/(i+1) :.3f}')
  print(f'time taken to run: {time.time() - start}')
  epoch_loss.append(round(ep_loss.item()/(i+1),2))

Epoch no. 1
Loss: 0.205, Accuracy: 17.955
time taken to run: 5501.818663358688
Epoch no. 2
Loss: 0.198, Accuracy: 17.968
time taken to run: 85.37167525291443
Epoch no. 3
Loss: 0.197, Accuracy: 17.968
time taken to run: 84.40736436843872
Epoch no. 4
Loss: 0.197, Accuracy: 17.968
time taken to run: 83.40253758430481
Epoch no. 5
Loss: 0.196, Accuracy: 17.968
time taken to run: 82.6618423461914
Epoch no. 6
Loss: 0.196, Accuracy: 17.968
time taken to run: 81.85242629051208
Epoch no. 7
Loss: 0.196, Accuracy: 17.968
time taken to run: 84.27642512321472
Epoch no. 8
Loss: 0.196, Accuracy: 17.968
time taken to run: 84.80474400520325
Epoch no. 9
Loss: 0.195, Accuracy: 17.968
time taken to run: 85.07283163070679
Epoch no. 10
Loss: 0.196, Accuracy: 17.968
time taken to run: 83.63415455818176
Epoch no. 11
Loss: 0.196, Accuracy: 17.968
time taken to run: 79.14959454536438
Epoch no. 12
Loss: 0.195, Accuracy: 17.968
time taken to run: 78.74146771430969
Epoch no. 13
Loss: 0.195, Accuracy: 17.968
time ta

In [ ]:
torch.save(model_alexnet.state_dict(), data_path+'/trained_alexnet_with_aug_wd.pth')

In [ ]:
plt.figure(figsize=(15,4))
plt.title('Training Loss per epoch')
plt.ylabel('Training Loss')
plt.xlabel('Epochs')
plt.plot(epoch_loss)
plt.grid()

In [ ]:
model_alexnet = AlexNet()
model_alexnet.load_state_dict(torch.load(data_path+'/trained_alexnet_with_aug_wd.pth'))
model_alexnet.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
criterion = nn.BCELoss()
# model_alexnet.to(dev)
model_alexnet.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

### Model Validation on validation set data

In [ ]:
model_alexnet.cpu()
model_alexnet.eval()

pred_results = np.array(range(1,20)).reshape(1,19)
target_results = np.array(range(1,20)).reshape(1,19)

validation_loss = 0.0
validation_accuracy = 0.0

for step, (inputs, label) in enumerate(validation_loader):
  print(step)
  inputs = torch.Tensor.cpu(inputs)#.to(dev)
  label = torch.Tensor.cpu(label)#.to(dev)

  # Make predictions
  y_pred = model_alexnet(inputs)
  preds_rounded = torch.round(y_pred) #anything greater than prob > 0.5 converted to predicted label
  preds_np = torch.Tensor.cpu(preds_rounded).detach().numpy()
  targets_np = torch.Tensor.cpu(label).detach().numpy()

  # Store predictions
  pred_results = np.concatenate((pred_results, preds_np), axis=0)
  target_results = np.concatenate((target_results, targets_np), axis=0)

  acc = accuracy(y_pred.cpu(), label.cpu())
  validation_loss += criterion(y_pred.float(), label.float())
  validation_accuracy += acc

average_validation_loss = validation_loss/(len(validation_loader))
average_validation_accuracy = validation_accuracy/(len(validation_loader))

print(f'Average validation loss: {average_validation_loss}')
print(f'Average validation accuracy: {average_validation_accuracy}')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
Average validation loss: 0.1897120624780655
Average validation accuracy: 17.971409574468087


### Model evaluation
- Classification Report
- Confusion Matrix for all classes

In [ ]:
classes = [str(i).zfill(2) for i in range(1,20)]
print(class_names, classes)
print(classification_report(target_results[1:], pred_results[1:], target_names=classes , zero_division=1))

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19']
              precision    recall  f1-score   support

          01       0.76      1.00      0.86      4555
          02       1.00      0.00      0.00       233
          03       1.00      0.00      0.00       884
          04       1.00      0.00      0.00       249
          05       1.00      0.00      0.00       231
          06       1.00      0.00      0.00       277
          07       1.00      0.00      0.00       238
          08       1.00      0.00      0.00       439
          09       1.00      0.00      0.00       194
          10       1.00      0.00      0.00       310
          11       1.00      0.00      0.00       111
          12       1.00      1.00      1.00         0
          13       1.00      0.00      0.00       101
          14       1.00      0.00      0.00        53
          15   

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, coverage_error
coverage_error(target_results[1:], pred_results[1:])

10.385564260710119

In [ ]:
# multilabel_confusion_matrix(target_results[1:], pred_results[1:])

multi_matrix = multilabel_confusion_matrix(target_results[1:], pred_results[1:])

labels = ["".join("c" + str(i)) for i in range(1, 20)]

def display_heatmap(confusion_matrix, axes, class_labels, class_names, fontsize=16):

    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names,
    )
    heatmap = sns.heatmap(df_cm, annot=True, fmt="d", cbar=False, ax=axes)
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    axes.set_ylabel('True label')
    axes.set_xlabel('Predicted label')
    axes.set_title("Confusion Matrix for - " + class_labels)


fig, ax = plt.subplots(4, 5, figsize=(15, 10))
for axes, cfs_matrix, label in zip(ax.flatten(), multi_matrix, labels):
        display_heatmap(cfs_matrix, axes, label, ["N", "Y"])
    
fig.tight_layout()
plt.show()

array([[[   0, 1444],
        [   0, 4555]],

       [[5766,    0],
        [ 233,    0]],

       [[5115,    0],
        [ 884,    0]],

       [[5750,    0],
        [ 249,    0]],

       [[5768,    0],
        [ 231,    0]],

       [[5722,    0],
        [ 277,    0]],

       [[5761,    0],
        [ 238,    0]],

       [[5560,    0],
        [ 439,    0]],

       [[5805,    0],
        [ 194,    0]],

       [[5689,    0],
        [ 310,    0]],

       [[5888,    0],
        [ 111,    0]],

       [[5999,    0],
        [   0,    0]],

       [[5898,    0],
        [ 101,    0]],

       [[5946,    0],
        [  53,    0]],

       [[5602,    0],
        [ 397,    0]],

       [[5786,    0],
        [ 213,    0]],

       [[5730,    0],
        [ 269,    0]],

       [[5682,    0],
        [ 317,    0]],

       [[5788,    0],
        [ 211,    0]]])

## Test Prediction
- Initialize test loader
- Run and save predictions

In [ ]:
## Make test predictions
model_alexnet.eval()
pred_results_test = np.array(range(1,20)).reshape(1,19)
images = []
predictions = []

for step, (inputs) in enumerate(test_loader):
    if step % 100 == 0:
      print(step)
    images.append(test_dataset.annotations.ImageID[step])
    inputs = torch.Tensor.cpu(inputs)#.to(dev)
    inputs = inputs.unsqueeze(0)

    y_pred = model_alexnet(inputs)

    preds_rounded = torch.round(y_pred) #anything greater than prob > 0.5 converted to predicted label
    preds_np = torch.Tensor.cpu(preds_rounded).detach().numpy()

    preds_classes = class_names[preds_np[0] == 1.0]
    preds_string = [str(int) for int in preds_classes]
    
    predictions.append(' '.join(preds_string))
    pred_results_test = np.concatenate((pred_results_test, preds_np), axis=0)

prediction_upload = pd.DataFrame(zip(images, predictions), columns = ['ImageID', 'Labels'])
prediction_upload.to_csv(data_path + '/alexnet_aug_wd_test_predictions.csv', index = False)
prediction_upload.head()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


,ImageID,Labels
0,30000.jpg,1
1,30001.jpg,1
2,30002.jpg,1
3,30003.jpg,1
4,30004.jpg,1
